# Manuale

0 [Introduzione](1_Introduzione.html)

1 [Il linguaggio HOL](3_Liguaggio_HOL.html)

2 [Appendice A5 - Regole d'inferenza](A5_Inference_Rules.html)

3 [Appendice B1 - Teorie](B1_Teorie.html)

4 [Appendice B2 - Teoremi](B2_Teoremi.html)

[Teorie.xlsx](Teorie.xlsx)

